In [ ]:
!pip install kaggle -q

In [1]:
# from google.colab import files
# files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#!/bin/bash
!kaggle models instances versions   download google/gemma-2/transformers/gemma-2-9b/2

100% 32.0G/32.0G [04:48<00:00, 136MB/s]
100% 32.0G/32.0G [04:48<00:00, 119MB/s]
/content/gemma-2.tar.gz

In [ ]:
!mkdir -p ./data/model
!tar -xzvf gemma-2.tar.gz -C ./data/model
!ls ./data/model

.gitattributes
README.md
config.json
generation_config.json
model-00001-of-00008.safetensors
model-00002-of-00008.safetensors
model-00003-of-00008.safetensors
model-00004-of-00008.safetensors
model-00005-of-00008.safetensors
model-00006-of-00008.safetensors
model-00007-of-00008.safetensors
model-00008-of-00008.safetensors
model.safetensors.index.json
special_tokens_map.json
tokenizer.json
tokenizer.model
tokenizer_config.json
transformers/
transformers/transformers-4.42.0.dev0-py3-none-any.whl
config.json			  model-00005-of-00008.safetensors  special_tokens_map.json
generation_config.json		  model-00006-of-00008.safetensors  tokenizer_config.json
model-00001-of-00008.safetensors  model-00007-of-00008.safetensors  tokenizer.json
model-00002-of-00008.safetensors  model-00008-of-00008.safetensors  tokenizer.model
model-00003-of-00008.safetensors  model.safetensors.index.json	    transformers
model-00004-of-00008.safetensors  README.md


In [ ]:
print("notebook started..")
!pip install -q transformers==4.46.3
# !pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
print("pip installs done!")

import numpy as np
import pandas as pd
import random
import math

notebook started..
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import transformers
import accelerate
import bitsandbytes

print(transformers.__version__)
print(accelerate.__version__)
print(bitsandbytes.__version__)


4.46.3
1.3.0
0.45.1


# Reload prior submission.csv and optimize from that!!!
* Set `reuse_last_submission = False` to start fresh
* Specify any samples we don't think need further optimization
* Specify any samples we want to fully shuffle between optimization rounds (stuck at local optimum)

In [ ]:
# Set to False to do a "clean" run
reuse_last_submission = True

if reuse_last_submission:
    samples = pd.read_csv("/content/submission (57).csv")
else:
    samples = pd.read_csv("/content/submission (57).csv")

# If we think some items are well-optimized - including them here will skip them
skip = [0, 1, 2, 3, 5]

# If some samples are badly stuck at local optimum - we fully shuffle between rounds
shuffle_between_cycles = []

# Batch-enabled scorer
* Re-enabled 8-bit quantization
* Make use of T4x2 setup
* Credit: https://www.kaggle.com/code/cdeotte/brute-force-first-sample-perplexity-470

In [ ]:
import gc
import os
from math import exp
from collections import Counter
from typing import List, Optional, Union

import numpy as np
import pandas as pd
import transformers
import torch

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
PAD_TOKEN_LABEL_ID = torch.nn.CrossEntropyLoss().ignore_index
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class ParticipantVisibleError(Exception):
    pass


def score(
    solution: pd.DataFrame,
    submission: pd.DataFrame,
    row_id_column_name: str,
    model_path: str = '/kaggle/input/gemma-2/transformers/gemma-2-9b/2',
    load_in_8bit: bool = True,
    clear_mem: bool = False,
) -> float:
    """
    Calculates the mean perplexity of submitted text permutations compared to an original text.

    Parameters
    ----------
    solution : DataFrame
        DataFrame containing the original text in a column named 'text'.
        Includes a row ID column specified by `row_id_column_name`.

    submission : DataFrame
        DataFrame containing the permuted text in a column named 'text'.
        Must have the same row IDs as the solution.
        Includes a row ID column specified by `row_id_column_name`.

    row_id_column_name : str
        Name of the column containing row IDs.
        Ensures aligned comparison between solution and submission.

    model_path : str
        Path to the serialized LLM.

    clear_mem : bool
        Clear GPU memory after scoring by clearing the CUDA cache.
        Useful for testing.

    Returns
    -------
    float
        The mean perplexity score. Lower is better.

    Raises
    ------
    ParticipantVisibleError
        If the submission format is invalid or submitted strings are not valid permutations.

    Examples
    --------
    >>> import pandas as pd
    >>> model_path = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"
    >>> solution = pd.DataFrame({
    ...     'id': [0, 1],
    ...     'text': ["this is a normal english sentence", "the quick brown fox jumps over the lazy dog"]
    ... })
    >>> submission = pd.DataFrame({
    ...     'id': [0, 1],
    ...     'text': ["sentence english normal a is this", "lazy the over jumps fox brown quick the dog"]
    ... })
    >>> score(solution, submission, 'id', model_path=model_path, clear_mem=True) > 0
    True
    """
    # Check that each submitted string is a permutation of the solution string
    sol_counts = solution.loc[:, 'text'].str.split().apply(Counter)
    sub_counts = submission.loc[:, 'text'].str.split().apply(Counter)
    invalid_mask = sol_counts != sub_counts
    if invalid_mask.any():
        raise ParticipantVisibleError(
            'At least one submitted string is not a valid permutation of the solution string.'
        )

    # Calculate perplexity for the submitted strings
    sub_strings = [
        ' '.join(s.split()) for s in submission['text'].tolist()
    ]  # Split and rejoin to normalize whitespace
    scorer = PerplexityCalculator(
        model_path=model_path,
        load_in_8bit=load_in_8bit,
    )  # Initialize the perplexity calculator with a pre-trained model
    perplexities = scorer.get_perplexity(
        sub_strings
    )  # Calculate perplexity for each submitted string

    if clear_mem:
        # Just move on if it fails. Not essential if we have the score.
        try:
            scorer.clear_gpu_memory()
        except:
            print('GPU memory clearing failed.')

    return float(np.mean(perplexities))


class PerplexityCalculator:
    """
    Calculates perplexity of text using a pre-trained language model.

    Adapted from https://github.com/asahi417/lmppl/blob/main/lmppl/ppl_recurrent_lm.py

    Parameters
    ----------
    model_path : str
        Path to the pre-trained language model

    load_in_8bit : bool, default=False
        Use 8-bit quantization for the model. Requires CUDA.

    device_map : str, default="auto"
        Device mapping for the model.
    """

    def __init__(
        self,
        model_path: str,
        load_in_8bit: bool = False,
        device_map: str = 'auto',
    ):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_path,padding_side="right")
        # Configure model loading based on quantization setting and device availability
        if load_in_8bit:
            if DEVICE.type != 'cuda':
                raise ValueError('8-bit quantization requires CUDA device')

            #quantization_config = transformers.BitsAndBytesConfig(load_in_8bit=True)
            #quantization_config = transformers.BitsAndBytesConfig(load_in_4bit=True)

            quantization_config = transformers.BitsAndBytesConfig(
                load_in_4bit = True,
                bnb_4bit_quant_type = "fp4", #fp4 nf4
                bnb_4bit_use_double_quant = False,
                bnb_4bit_compute_dtype=torch.float16,
            )

            self.model = transformers.AutoModelForCausalLM.from_pretrained(
                model_path,
                quantization_config=quantization_config,
                device_map=device_map,
            )
        else:
            self.model = transformers.AutoModelForCausalLM.from_pretrained(
                model_path,
                torch_dtype=torch.float16 if DEVICE.type == 'cuda' else torch.float32,
                device_map=device_map,
            )

        self.loss_fct = torch.nn.CrossEntropyLoss(reduction='none')

        self.model.eval()
        #if not load_in_8bit:
        #    self.model.to(DEVICE)  # Explicitly move the model to the device

    def get_perplexity(
        self, input_texts: Union[str, List[str]], batch_size: 32
    ) -> Union[float, List[float]]:
        """
        Calculates the perplexity of given texts.

        Parameters
        ----------
        input_texts : str or list of str
            A single string or a list of strings.

        batch_size : int, default=None
            Batch size for processing. Defaults to the number of input texts.

        verbose : bool, default=False
            Display progress bar.

        Returns
        -------
        float or list of float
            A single perplexity value if input is a single string,
            or a list of perplexity values if input is a list of strings.

        Examples
        --------
        >>> import pandas as pd
        >>> model_path = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"
        >>> scorer = PerplexityCalculator(model_path=model_path)

        >>> submission = pd.DataFrame({
        ...     'id': [0, 1, 2],
        ...     'text': ["this is a normal english sentence", "thsi is a slihgtly misspelled zr4g sentense", "the quick brown fox jumps over the lazy dog"]
        ... })
        >>> perplexities = scorer.get_perplexity(submission["text"].tolist())
        >>> perplexities[0] < perplexities[1]
        True
        >>> perplexities[2] < perplexities[0]
        True

        >>> perplexities = scorer.get_perplexity(["this is a sentence", "another sentence"])
        >>> all(p > 0 for p in perplexities)
        True

        >>> scorer.clear_gpu_memory()
        """
        single_input = isinstance(input_texts, str)
        input_texts = [input_texts] if single_input else input_texts

        loss_list = []

        batches = len(input_texts)//batch_size + (len(input_texts)%batch_size != 0)
        for j in range(batches):

            a = j*batch_size
            b = (j+1)*batch_size
            input_batch = input_texts[a:b]

            with torch.no_grad():

                # Explicitly add sequence boundary tokens to the text
                text_with_special = [f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}" for text in input_batch]

                # Tokenize
                model_inputs = self.tokenizer(
                    text_with_special,
                    return_tensors='pt',
                    add_special_tokens=False,
                    padding=True
                )

                if 'token_type_ids' in model_inputs:
                    model_inputs.pop('token_type_ids')

                model_inputs = {k: v.to(DEVICE) for k, v in model_inputs.items()}

                # Get model output
                output = self.model(**model_inputs, use_cache=False)
                logits = output['logits']

                label = model_inputs['input_ids']
                label[label == self.tokenizer.pad_token_id] = PAD_TOKEN_LABEL_ID

                # Shift logits and labels for calculating loss
                shift_logits = logits[..., :-1, :].contiguous()  # Drop last prediction
                shift_labels = label[..., 1:].contiguous()  # Drop first input

                # Calculate token-wise loss
                loss = self.loss_fct(
                    shift_logits.view(-1, shift_logits.size(-1)),
                    shift_labels.view(-1)
                )

                loss = loss.view(len(logits), -1)
                valid_length = (shift_labels != PAD_TOKEN_LABEL_ID).sum(dim=-1)
                loss = torch.sum(loss, -1) / valid_length

                loss_list += loss.cpu().tolist()

                # Debug output
                #print(f"\nProcessing: '{text}'")
                #print(f"With special tokens: '{text_with_special}'")
                #print(f"Input tokens: {model_inputs['input_ids'][0].tolist()}")
                #print(f"Target tokens: {shift_labels[0].tolist()}")
                #print(f"Input decoded: {self.tokenizer.decode(model_inputs['input_ids'][0])}")
                #print(f"Target decoded: {self.tokenizer.decode(shift_labels[0])}")
                #print(f"Individual losses: {loss.tolist()}")
                #print(f"Average loss: {sequence_loss.item():.4f}")

        ppl = [exp(i) for i in loss_list]

        # print("\nFinal perplexities:")
        # for text, perp in zip(input_texts, ppl):
        #     print(f"Text: '{text}'")
        #     print(f"Perplexity: {perp:.2f}")

        return ppl[0] if single_input else ppl

    def clear_gpu_memory(self) -> None:
        """Clears GPU memory by deleting references and emptying caches."""
        if not torch.cuda.is_available():
            return

        # Delete model and tokenizer if they exist
        if hasattr(self, 'model'):
            del self.model
        if hasattr(self, 'tokenizer'):
            del self.tokenizer

        # Run garbage collection
        gc.collect()

        # Clear CUDA cache and reset memory stats
        with DEVICE:
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            torch.cuda.reset_peak_memory_stats()

# Load custom scorer

In [ ]:
#gemma-2-9b (competition scoring metric)
scorer = PerplexityCalculator('/content/data/model')

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

# Get our starting scores
* On initial dataset scorer returns some NaN's - we'll account for that..
* Since we've re-run this notebook a few times - further re-runs may not improve the score much...

In [ ]:
%%time
# Get actual mean value
scores = []
for row in range(len(samples)):
    score = scorer.get_perplexity(samples.iloc[row].text, batch_size=1)
    print(samples.iloc[row].text)
    print(f"Score: {score:.2f}\n")
    scores.append(score)

print(f"Starting mean score: {np.mean(scores):.2f}")

reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament
Score: 467.99

reindeer sleep walk the night and drive mistletoe scrooge laugh chimney jump elf bake gingerbread family give advent fireplace ornament
Score: 424.44

sleigh yuletide beard carol cheer chimney decorations gifts grinch holiday holly jingle magi naughty nice nutcracker ornament polar workshop stocking
Score: 299.80

sleigh of the magi yuletide cheer is unwrap gifts and eat cheer holiday decorations holly jingle relax sing carol visit workshop grinch naughty nice chimney stocking ornament nutcracker polar beard
Score: 202.07

from and of to the as in that it we with not you have milk chocolate candy fruitcake eggnog peppermint season greeting card wrapping paper bow toy doll game puzzle cookie snowglobe fireplace candle wreath poinsettia angel star wish dream night wonder believe hope joy peace merry hohoho kaggle workshop
Score: 70.38

from and and as we and have the in is it of not that th

In [ ]:
# @title old version
# import random
# import math
# from typing import List, Tuple

# def partial_shuffle(sequence: List[str], fraction: float = 0.1) -> None:
#     """
#     Shuffle only a fraction of the sequence in-place.

#     Args:
#         sequence (List[str]): The list of words to shuffle partially.
#         fraction (float): The fraction of indices to randomly shuffle.
#     """
#     length = len(sequence)
#     subset_size = int(round(length * fraction))
#     if subset_size <= 0:
#         return  # No shuffle if fraction is too small or list is too short

#     # Randomly sample the indices to shuffle
#     indices_to_shuffle = random.sample(range(length), subset_size)

#     # Extract the items in those indices
#     items_to_shuffle = [sequence[i] for i in indices_to_shuffle]

#     # Shuffle the subset
#     random.shuffle(items_to_shuffle)

#     # Place them back
#     for i, idx in enumerate(indices_to_shuffle):
#         sequence[idx] = items_to_shuffle[i]

# def local_search(current: List[str], current_score: float, scorer) -> Tuple[List[str], float]:
#     """
#     Repeatedly attempt adjacent swaps in the current sequence.
#     If any swap improves perplexity, accept it and restart from the beginning.
#     Continue until no improvement is found.
#     """
#     improved = True
#     while improved:
#         improved = False
#         for i in range(len(current) - 1):
#             # Try swapping adjacent words
#             neighbor = current.copy()
#             neighbor[i], neighbor[i+1] = neighbor[i+1], neighbor[i]

#             new_score = scorer.get_perplexity(" ".join(neighbor), batch_size=1)
#             if new_score < current_score:
#                 # Accept this local improvement
#                 current = neighbor
#                 current_score = new_score
#                 improved = True
#                 break  # Restart from the beginning once an improvement is found
#     return current, current_score

# def generate_neighbors(current: List[str], strategy: str = "swap") -> List[str]:
#     """
#     Generate a neighbor of the current sequence based on the specified strategy.

#     Args:
#         current (List[str]): The current sequence of words.
#         strategy (str): The strategy for generating a neighbor. Options are:
#             - "swap": Swap two random words.
#             - "reverse": Reverse a random subsequence.
#             - "shuffle_subsequence": Shuffle a random subsequence.
#             - "shift": Rotate a random subsequence to the left or right.

#     Returns:
#         List[str]: A new sequence generated by the specified strategy.
#     """
#     neighbor = current.copy()

#     if strategy == "swap":
#         # Swap two random words
#         i, j = random.sample(range(len(neighbor)), 2)
#         neighbor[i], neighbor[j] = neighbor[j], neighbor[i]

#     elif strategy == "reverse":
#         # Reverse a random subsequence
#         i, j = sorted(random.sample(range(len(neighbor)), 2))
#         neighbor[i:j+1] = reversed(neighbor[i:j+1])

#     elif strategy == "shuffle_subsequence":
#         # Shuffle a random subsequence
#         i, j = sorted(random.sample(range(len(neighbor)), 2))
#         subsequence = neighbor[i:j+1]
#         random.shuffle(subsequence)
#         neighbor[i:j+1] = subsequence

#     elif strategy == "shift":
#         # Rotate a random subsequence left or right
#         i, j = sorted(random.sample(range(len(neighbor)), 2))
#         subsequence = neighbor[i:j+1]
#         if random.choice([True, False]):  # Randomly decide direction
#             # Rotate left
#             subsequence = subsequence[1:] + subsequence[:1]
#         else:
#             # Rotate right
#             subsequence = subsequence[-1:] + subsequence[:-1]
#         neighbor[i:j+1] = subsequence

#     else:
#         raise ValueError(f"Unknown strategy: {strategy}")

#     return neighbor

# def simulated_annealing_optimize(
#         text: str,
#         scorer,
#         temp_start: float = 3.0,
#         temp_end: float = 0.02,
#         cooling_rate: float = 0.99,
#         samples_per_temp: int = 250,
#         max_batch_size: int = 64,
#         max_words_per_batch: int = 640,
#         reheat_cycles: int = 8,
#         low_temp_samples_after_improve: int = 1000,
#         temp_start_increase_per_cycle: float = 0.5,
#         shuffle_chance_between_cycles: float = 1.0,
#         verbose: bool = False,
#         initial_sequence: List[str] = None,
#         perplexity_threshold: float = 80.0,
#         terminate_on_threshold: bool = False
#     ) -> Tuple[str, float]:
#     """
#     Optimize the ordering of words in `text` to minimize perplexity via simulated annealing,
#     with an added local search phase after each accepted move.

#     This version partially shuffles 30% of the sequence at the start (if no initial_sequence)
#     and after each cycle (instead of a full shuffle).
#     """

#     # 1. Start from the *initial_sequence* if provided, else the original sequence
#     words = text.split()
#     if initial_sequence is not None:
#         current = initial_sequence.copy()
#     else:
#         current = words.copy()
#         # Partially shuffle 30% of the text at the beginning
#         partial_shuffle(current, 0.1)

#     current_text = ' '.join(current)

#     # Compute perplexity of the current sequence
#     current_score = scorer.get_perplexity(current_text, batch_size=1)
#     attempts = 0
#     while math.isnan(current_score) and attempts < 10:
#         # If the score is NaN, try partial shuffling again
#         partial_shuffle(current, 0.1)
#         current_text = ' '.join(current)
#         current_score = scorer.get_perplexity(current_text, batch_size=1)
#         attempts += 1

#     if math.isnan(current_score):
#         raise ValueError("Unable to find a valid initial permutation (NaN perplexity).")

#     best = current.copy()
#     best_score = current_score

#     print(f"Initial score: {current_score:.2f}")

#     # Prepare batch/neighbor logic
#     max_neighbors = max_words_per_batch // len(words) if len(words) > 0 else 1
#     actual_batch_size = min(max_batch_size, max_neighbors)
#     batches_per_temp = (samples_per_temp + actual_batch_size - 1) // actual_batch_size

#     # Statistics trackers
#     total_moves = 0
#     accepted_moves = 0
#     improved_moves = 0
#     rejected_moves = 0

#     def print_stats():
#         if total_moves == 0:
#             return
#         acceptance_rate = accepted_moves / total_moves
#         improvement_rate = improved_moves / total_moves
#         print(
#             f"  [Stats] Moves={total_moves}, Accepted={accepted_moves}, Rejected={rejected_moves}, "
#             f"AcceptanceRate={acceptance_rate:.3f}, ImprovementRate={improvement_rate:.3f}, "
#             f"CurrentScore={current_score:.2f}, BestScore={best_score:.2f}"
#         )

#     reject_counter = 0
#     initial_temp_value = temp_start

#     # Main reheat cycles
#     for cycle in range(reheat_cycles):
#         temp = temp_start
#         scaling_factor = 1.0
#         print(f"\n=== Cycle {cycle + 1}/{reheat_cycles} - Starting temp: {temp:.2f} ===")

#         remaining_low_temp_steps = 0
#         stored_temp = None

#         # Loop while temp > temp_end or we are in low-temp mode
#         while temp > temp_end or remaining_low_temp_steps > 0:
#             samples_processed = 0

#             for batch in range(batches_per_temp):
#                 remaining_samples = samples_per_temp - samples_processed
#                 current_batch_size = min(actual_batch_size, remaining_samples)
#                 if current_batch_size <= 0:
#                     break

#                 # Generate and score neighbors
#                 neighbors = []
#                 neighbor_texts = []
#                 for _ in range(current_batch_size):
#                     neighbor = generate_neighbors(
#                         current,
#                         strategy=random.choice(["swap", "shift", "reverse", "shuffle_subsequence"])
#                     )
#                     neighbors.append(neighbor)
#                     neighbor_texts.append(' '.join(neighbor))

#                 neighbor_scores = scorer.get_perplexity(neighbor_texts, batch_size=current_batch_size)

#                 for neighbor, neighbor_score in zip(neighbors, neighbor_scores):
#                     samples_processed += 1
#                     total_moves += 1
#                     if math.isnan(neighbor_score):
#                         continue

#                     delta = neighbor_score - current_score
#                     current_temp = temp_end if remaining_low_temp_steps > 0 else temp
#                     scaling_factor = max(1.0, temp / initial_temp_value)

#                     # Metropolis acceptance criterion
#                     acceptance_probability = math.exp(-delta / (current_temp * scaling_factor)) if delta > 0 else 1.0
#                     if delta < 0 or random.random() < acceptance_probability:
#                         # ACCEPT
#                         accepted_moves += 1
#                         current = neighbor
#                         current_score = neighbor_score

#                         # --- LOCAL SEARCH INTENSIFICATION ---
#                         current, current_score = local_search(current, current_score, scorer)

#                         # Check if local search improved the solution
#                         if current_score < best_score:
#                             best = current.copy()
#                             best_score = current_score
#                             improved_moves += 1

#                             # Store temperature if not in low-temp mode
#                             if remaining_low_temp_steps == 0:
#                                 stored_temp = temp
#                             remaining_low_temp_steps = low_temp_samples_after_improve
#                             print(">", end="", flush=True)  # improvement
#                             if best_score < perplexity_threshold:
#                                 print(f"\n*** Perplexity threshold reached: {best_score:.2f} ***")
#                                 print(f"Best sequence: {' '.join(best)}")
#                                 if terminate_on_threshold:
#                                     print("Terminating optimization early as threshold was met.")
#                                     return ' '.join(best), best_score
#                         else:
#                             print("<", end="", flush=True)  # accepted but not improved
#                     else:
#                         # REJECT
#                         reject_counter += 1
#                         rejected_moves += 1
#                         if reject_counter % 10 == 0:
#                             print("-", end="", flush=True)

#             # Print intermediate stats
#             print_stats()

#             # Temperature or low-temp mode management
#             if remaining_low_temp_steps > 0:
#                 remaining_low_temp_steps -= samples_per_temp
#                 if verbose:
#                     print(f"\nLow-temp mode ({remaining_low_temp_steps} steps remaining). "
#                           f"Current={current_score:.2f}, Best={best_score:.2f}")
#                 if remaining_low_temp_steps <= 0 and stored_temp is not None:
#                     temp = stored_temp  # Return to the stored temperature
#             else:
#                 temp *= cooling_rate
#                 if verbose:
#                     print(f"\nCooling down: Temp={temp:.2f}, Current={current_score:.2f}, Best={best_score:.2f}")

#         print(f"\nEnd of cycle {cycle + 1}, best score so far: {best_score:.2f}")

#         # Partially shuffle 30% of the current sequence between cycles
#         current = best.copy()
#         partial_shuffle(current, 0.1)
#         current_score = scorer.get_perplexity(' '.join(current), batch_size=1)
#         print(f"New partial-shuffled score: {current_score:.2f}")

#         # Increase starting temp for next cycle
#         temp_start += temp_start_increase_per_cycle

#     # Final output
#     print("\n=== Optimization finished ===")
#     print_stats()
#     print(f"Final best score: {best_score:.2f}")

#     return ' '.join(best), best_score

# def simulated_annealing_with_restarts(
#         text: str,
#         scorer,
#         num_restarts: int = 10,  # Number of random restarts
#         temp_start: float = 3.0,
#         temp_end: float = 0.02,
#         cooling_rate: float = 0.99,
#         samples_per_temp: int = 250,
#         max_batch_size: int = 64,
#         max_words_per_batch: int = 640,
#         reheat_cycles: int = 8,
#         low_temp_samples_after_improve: int = 1000,
#         temp_start_increase_per_cycle: float = 0.5,
#         shuffle_chance_between_cycles: float = 1.0,
#         verbose: bool = False,
#         perplexity_threshold: float = 80.0,
#         terminate_on_threshold: bool = False
#     ) -> Tuple[str, float]:
#     """
#     Perform simulated annealing optimization with multiple random restarts,
#     each time partially shuffling 30% of the text for the initial sequence.

#     Args:
#         text (str): Original text to shuffle.
#         scorer: An object or function that provides a `get_perplexity` method.
#         num_restarts (int): Number of random restarts to perform.
#         ... (other parameters as in `simulated_annealing_optimize`)

#     Returns:
#         (str, float): The best sequence of words found across all restarts, and its perplexity score.
#     """
#     best_overall = None
#     best_overall_score = float('inf')

#     words = text.split()

#     for restart in range(num_restarts):
#         print(f"\n--- Restart {restart + 1}/{num_restarts} ---")

#         # Generate a random initial sequence by partially shuffling 30% of the words
#         random_initial = words.copy()
#         partial_shuffle(random_initial, 0.1)

#         # Run simulated annealing optimization from this partial-shuffled sequence
#         optimized_sequence, optimized_score = simulated_annealing_optimize(
#             text=text,
#             scorer=scorer,
#             temp_start=temp_start,
#             temp_end=temp_end,
#             cooling_rate=cooling_rate,
#             samples_per_temp=samples_per_temp,
#             max_batch_size=max_batch_size,
#             max_words_per_batch=max_words_per_batch,
#             reheat_cycles=reheat_cycles,
#             low_temp_samples_after_improve=low_temp_samples_after_improve,
#             temp_start_increase_per_cycle=temp_start_increase_per_cycle,
#             shuffle_chance_between_cycles=shuffle_chance_between_cycles,
#             verbose=verbose,
#             initial_sequence=random_initial,
#             perplexity_threshold=perplexity_threshold,
#             terminate_on_threshold=terminate_on_threshold
#         )

#         print(f"Restart {restart + 1} best score: {optimized_score:.2f}")

#         # Update the overall best solution
#         if optimized_score < best_overall_score:
#             best_overall_score = optimized_score
#             best_overall = optimized_sequence

#     print("\n=== All Restarts Completed ===")
#     print(f"Best overall score: {best_overall_score:.2f}")

#     return best_overall, best_overall_score


In [ ]:
import random
import math
from typing import List, Tuple

def move_random_word_to_front(seq: List[str]) -> List[str]:
    """Pick a random word from seq and move it to the front."""
    if not seq:
        return seq
    neighbor = seq.copy()
    idx = random.randrange(len(neighbor))
    word = neighbor.pop(idx)
    neighbor.insert(0, word)
    return neighbor

def generate_neighbors(current: List[str], strategy: str = "swap", k: int = 8) -> List[str]:
    """
    Generate a neighbor of the current sequence based on the specified strategy.

    Args:
        current (List[str]): The current sequence of words.
        strategy (str): The strategy for generating a neighbor. Options are:
            - "swap"
            - "reverse"
            - "shuffle_subsequence"
            - "shift"
            - "partial_permutation"
            - "shift_then_partial_permutation"
            - "partial_permutation_k3"
            - "shift_then_partial_permutation_k3"
        k (int): Number of words to permute for partial_permutation-based strategies.

    Returns:
        List[str]: A new sequence generated by the specified strategy.
    """
    neighbor = current.copy()
    length = len(neighbor)

    if length < 2:
        return neighbor  # No valid mutation possible

    if strategy == "swap":
        i, j = random.sample(range(length), 2)
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]

    elif strategy == "reverse":
        i, j = sorted(random.sample(range(length), 2))
        neighbor[i:j+1] = reversed(neighbor[i:j+1])

    elif strategy == "shuffle_subsequence":
        i, j = sorted(random.sample(range(length), 2))
        subsequence = neighbor[i:j+1]
        random.shuffle(subsequence)
        neighbor[i:j+1] = subsequence

    elif strategy == "shift":
        i, j = sorted(random.sample(range(length), 2))
        subsequence = neighbor[i:j+1]
        if random.choice([True, False]):  # Rotate left
            subsequence = subsequence[1:] + subsequence[:1]
        else:  # Rotate right
            subsequence = subsequence[-1:] + subsequence[:-1]
        neighbor[i:j+1] = subsequence

    elif strategy == "partial_permutation":
        indices = random.sample(range(length), min(k, length))
        permuted_values = random.sample([neighbor[i] for i in indices], len(indices))
        for idx, new_val in zip(indices, permuted_values):
            neighbor[idx] = new_val

    elif strategy == "shift_then_partial_permutation":
        # Step 1: Shift a random subsequence
        i, j = sorted(random.sample(range(length), 2))
        subsequence = neighbor[i:j+1]
        if random.choice([True, False]):  # Rotate left
            subsequence = subsequence[1:] + subsequence[:1]
        else:  # Rotate right
            subsequence = subsequence[-1:] + subsequence[:-1]
        neighbor[i:j+1] = subsequence

        # Step 2: Apply partial permutation
        indices = random.sample(range(length), min(k, length))
        permuted_values = random.sample([neighbor[i] for i in indices], len(indices))
        for idx, new_val in zip(indices, permuted_values):
            neighbor[idx] = new_val

    elif strategy == "partial_permutation_k3":
        k_fixed = 3
        indices = random.sample(range(length), min(k_fixed, length))
        permuted_values = random.sample([neighbor[i] for i in indices], len(indices))
        for idx, new_val in zip(indices, permuted_values):
            neighbor[idx] = new_val

    elif strategy == "shift_then_partial_permutation_k3":
        # Step 1: Shift a random subsequence
        i, j = sorted(random.sample(range(length), 2))
        subsequence = neighbor[i:j+1]
        if random.choice([True, False]):
            subsequence = subsequence[1:] + subsequence[:1]
        else:
            subsequence = subsequence[-1:] + subsequence[:-1]
        neighbor[i:j+1] = subsequence

        # Step 2: Partial permutation with k=3
        k_fixed = 3
        indices = random.sample(range(length), min(k_fixed, length))
        permuted_values = random.sample([neighbor[i] for i in indices], len(indices))
        for idx, new_val in zip(indices, permuted_values):
            neighbor[idx] = new_val

    else:
        raise ValueError(f"Unknown strategy: {strategy}")

    return neighbor


def simulated_annealing_optimize(
        text: str,
        scorer,
        temp_start: float = 3.0,
        temp_end: float = 0.02,
        cooling_rate: float = 0.99,
        samples_per_temp: int = 250,
        max_batch_size: int = 64,
        max_words_per_batch: int = 640,
        reheat_cycles: int = 8,
        low_temp_samples_after_improve: int = 1000,
        temp_start_increase_per_cycle: float = 0.5,
        shuffle_chance_between_cycles: float = 1.0,
        verbose: bool = False,
        initial_sequence: List[str] = None,
        perplexity_threshold: float = 80.0,
        terminate_on_threshold: bool = False
    ) -> Tuple[str, float]:
    """
    Optimize the ordering of words in text to minimize perplexity via simulated annealing.

    In this version, we apply a 'move_random_word_to_front' perturbation
    before each reheat cycle, then start from that perturbed sequence.
    """
    # Split the text into words
    words = text.split()
    if initial_sequence is not None:
        current = initial_sequence.copy()
    else:
        current = words.copy()

    # Score the initial sequence
    current_text = ' '.join(current)
    current_score = scorer.get_perplexity(current_text, batch_size=1)
    attempts = 0
    while math.isnan(current_score) and attempts < 10:
        # If the score is NaN, shuffle to see if we get a valid perplexity
        random.shuffle(current)
        current_text = ' '.join(current)
        current_score = scorer.get_perplexity(current_text, batch_size=1)
        attempts += 1

    if math.isnan(current_score):
        raise ValueError("Unable to find a valid initial permutation (NaN perplexity).")

    # Keep track of the overall best within this run
    best = current.copy()
    best_score = current_score

    print(f"Initial score: {current_score:.2f}")

    # Weighted strategies (giving higher probability to partial_permutation_k3, etc.)
    weighted_strategies = [
        ("swap", 1),
        ("shift", 1),
        ("reverse", 1),
        ("shuffle_subsequence", 1),
        ("partial_permutation", 1),
        ("shift_then_partial_permutation", 1),
        ("partial_permutation_k3", 3),
        ("shift_then_partial_permutation_k3", 3),
    ]

    # Prepare batch/neighbor logic
    max_neighbors = max_words_per_batch // len(words) if len(words) != 0 else 1
    if max_neighbors == 0:
        max_neighbors = 1
    actual_batch_size = min(max_batch_size, max_neighbors)
    if actual_batch_size == 0:
        actual_batch_size = 1
    batches_per_temp = (samples_per_temp + actual_batch_size - 1) // actual_batch_size

    # Tracking stats
    total_moves = 0
    accepted_moves = 0
    improved_moves = 0
    rejected_moves = 0

    def print_stats():
        if total_moves == 0:
            return
        acceptance_rate = accepted_moves / total_moves
        improvement_rate = improved_moves / total_moves
        print(
            f"  [Stats] Moves={total_moves}, Accepted={accepted_moves}, Rejected={rejected_moves}, "
            f"AcceptanceRate={acceptance_rate:.3f}, ImprovementRate={improvement_rate:.3f}, "
            f"CurrentScore={current_score:.2f}, BestScore={best_score:.2f}"
        )

    reject_counter = 0
    initial_temp_value = temp_start

    # Main reheat cycles
    for cycle in range(reheat_cycles):
        # Before each cycle, apply the small perturbation to our current best
        best = move_random_word_to_front(best)
        current = best.copy()  # Start from that perturbed solution
        current_score = scorer.get_perplexity(' '.join(current), batch_size=1)

        temp = temp_start
        scaling_factor = 1.0

        print(f"\n=== Cycle {cycle + 1}/{reheat_cycles} - Starting temp: {temp:.2f} ===")
        print(f"Perturbed score: {current_score:.2f}")

        remaining_low_temp_steps = 0
        stored_temp = None

        # Loop while temp > temp_end OR in low-temp mode
        while temp > temp_end or remaining_low_temp_steps > 0:
            samples_processed = 0

            for _ in range(batches_per_temp):
                remaining_samples = samples_per_temp - samples_processed
                current_batch_size = min(actual_batch_size, remaining_samples)
                if current_batch_size <= 0:
                    break

                neighbors = []
                neighbor_texts = []

                for _ in range(current_batch_size):
                    population = [s for s, w in weighted_strategies]
                    weights = [w for s, w in weighted_strategies]
                    chosen_strategy = random.choices(population, weights=weights, k=1)[0]

                    neighbor = generate_neighbors(current, strategy=chosen_strategy, k=8)
                    neighbors.append(neighbor)
                    neighbor_texts.append(' '.join(neighbor))

                neighbor_scores = scorer.get_perplexity(neighbor_texts, batch_size=current_batch_size)

                for neighbor, neighbor_score in zip(neighbors, neighbor_scores):
                    samples_processed += 1
                    total_moves += 1
                    if math.isnan(neighbor_score):
                        continue  # Skip NaNs

                    delta = neighbor_score - current_score
                    current_temp = temp_end if remaining_low_temp_steps > 0 else temp
                    scaling_factor = max(1.0, temp / initial_temp_value)

                    # Metropolis acceptance criterion
                    if delta < 0:
                        acceptance_probability = 1.0
                    else:
                        acceptance_probability = math.exp(-delta / (current_temp * scaling_factor))

                    if random.random() < acceptance_probability:
                        # ACCEPT
                        accepted_moves += 1
                        current = neighbor
                        current_score = neighbor_score

                        # Check if it's an improvement over the best
                        if current_score < best_score:
                            best = current.copy()
                            best_score = current_score
                            improved_moves += 1

                            # If not already in low-temp mode, store temperature
                            if remaining_low_temp_steps == 0:
                                stored_temp = temp
                            remaining_low_temp_steps = low_temp_samples_after_improve
                            print(">", end="", flush=True)
                            if best_score < perplexity_threshold:
                                print(f"\n*** Perplexity threshold reached: {best_score:.2f} ***")
                                print(f"Best sequence: {' '.join(best)}")
                                if terminate_on_threshold:
                                    print("Terminating optimization early as threshold was met.")
                                    return ' '.join(best), best_score
                        else:
                            print("<", end="", flush=True)
                    else:
                        # REJECT
                        reject_counter += 1
                        rejected_moves += 1
                        if reject_counter % 10 == 0:
                            print("-", end="", flush=True)

            # Print intermediate stats
            print_stats()

            # Temperature / low-temp mode management
            if remaining_low_temp_steps > 0:
                remaining_low_temp_steps -= samples_per_temp
                if verbose:
                    print(f"\nLow-temp mode ({remaining_low_temp_steps} steps remaining). "
                          f"Current={current_score:.2f}, Best={best_score:.2f}")
                if remaining_low_temp_steps <= 0 and stored_temp is not None:
                    # Return to the stored temperature after low-temp mode
                    temp = stored_temp
            else:
                # Standard cooling
                temp *= cooling_rate
                if verbose:
                    print(f"\nCooling down: Temp={temp:.4f}, Current={current_score:.2f}, Best={best_score:.2f}")

        print(f"\nEnd of cycle {cycle + 1}, best score so far: {best_score:.2f}")

        # Optional shuffle between cycles
        if random.random() < shuffle_chance_between_cycles:
            random.shuffle(current)
            current_score = scorer.get_perplexity(' '.join(current), batch_size=1)
            print(f"New shuffled score (between cycles): {current_score:.2f}")

        # Increase starting temp for the next cycle
        temp_start += temp_start_increase_per_cycle

    # Final output
    print("\n=== Optimization finished ===")
    print_stats()
    print(f"Final best score: {best_score:.2f}")

    return ' '.join(best), best_score


# Run and Submit!

In [ ]:
import torch

# Clear GPU memory
torch.cuda.empty_cache()

# Force garbage collection to free up memory
import gc
gc.collect()


167

In [ ]:
# text = "from and of to the as in that it we with not you have milk chocolate candy fruitcake eggnog peppermint season greeting card wrapping paper bow toy doll game night puzzle cookie snowglobe star angel wreath poinsettia candle fireplace wish dream believe wonder hope joy peace merry hohoho kaggle workshop"

text = samples.loc[3, "text"]

# text = "from and of to the as in that it we with not you have milk chocolate candy fruitcake eggnog peppermint season greeting card wrapping paper bow toy doll game puzzle cookie snowglobe fireplace candle wreath poinsettia angel star night wish wonder dream believe hope joy peace merry hohoho kaggle workshop"

# Assuming `scorer` is an object or function that provides get_perplexity(...)
best_sequence, best_score = simulated_annealing_optimize(
    text=text,
    scorer=scorer,
    temp_start=0.01,
    temp_end=0.001,
    cooling_rate=0.99,
    samples_per_temp=250,
    max_batch_size=64,
    max_words_per_batch=640,
    reheat_cycles=8,
    low_temp_samples_after_improve=2000,
    temp_start_increase_per_cycle=0.005,
    shuffle_chance_between_cycles=0.0,
    verbose=True,              # For detailed logs
    perplexity_threshold=202.07,
    terminate_on_threshold=False
)

print(f"\nBest optimized sequence: {best_sequence}")
print(f"Best perplexity score: {best_score:.2f}")

Streaming output truncated to the last 5000 lines.
-------------------------  [Stats] Moves=228000, Accepted=6592, Rejected=221408, AcceptanceRate=0.029, ImprovementRate=0.000, CurrentScore=201.28, BestScore=201.28

Cooling down: Temp=0.0087, Current=201.28, Best=201.28
-------------------------  [Stats] Moves=228250, Accepted=6592, Rejected=221658, AcceptanceRate=0.029, ImprovementRate=0.000, CurrentScore=201.28, BestScore=201.28

Cooling down: Temp=0.0086, Current=201.28, Best=201.28
------------------------<-  [Stats] Moves=228500, Accepted=6593, Rejected=221907, AcceptanceRate=0.029, ImprovementRate=0.000, CurrentScore=201.28, BestScore=201.28

Cooling down: Temp=0.0085, Current=201.28, Best=201.28
-------------------------  [Stats] Moves=228750, Accepted=6593, Rejected=222157, AcceptanceRate=0.029, ImprovementRate=0.000, CurrentScore=201.28, BestScore=201.28

Cooling down: Temp=0.0084, Current=201.28, Best=201.28
------------------------<-  [Stats] Moves=229000, Accepted=6594, Rej